In [3]:
from langchain.chains import LLMChain
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate

bedrock = Bedrock(
  credentials_profile_name="default",
  model_id="anthropic.claude-v2"
)

# First Chain 
template = """ you are a playwright. when you get title of theater, your role is writing synopsis.
title : {title}
synopsis : 
"""

prompt = PromptTemplate(
  input_variables=["title"],
  template=template
)

chain1 = LLMChain(
  llm=bedrock,
  prompt=prompt
)

# Second Chain
template2 = """ You are a theater critic. Given a synopsis of a play, your job is to write a review of it.
synopsis : {synopsis}
review :
"""

prompt2 = PromptTemplate(
  input_variables=["synopsis"],
  template=template2
)

chain2 = LLMChain(
  llm=bedrock,
  prompt=prompt2
)

from langchain.chains import SimpleSequentialChain

#Connect two chain with SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

print(overall_chain.run("The Moon of Seoul"))




> Entering new SimpleSequentialChain chain...
 Here is a potential synopsis for a play titled "The Moon of Seoul":

It is the 1960s in Seoul, South Korea. Young-sook is a bright yet shy farm girl who dreams of becoming a famous singer in the city one day. When her childhood friend Min-jun returns from studying abroad, sparks fly between them. However, Min-jun must focus on taking over his father's successful business instead of pursuing his passion for art and music. 

When Young-sook auditions for a spot on a popular television singing competition, she wows the judges with her beautiful voice. As she advances through the competition rounds, her fame grows. Young-sook gets a chance to move to Seoul and sign a record deal, putting her dream within reach.

But the fame and fortune quickly go to Young-sook's head. She turns arrogant and loses touch with her humble roots. Min-jun tries to reconnect with Young-sook and remind her of who she really is, but she pushes him away. On the night

In [21]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores.faiss import FAISS

with open("../data/akazukin_all.txt", encoding='UTF8') as f:
  test_all = f.read()

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=300, chunk_overlap=20)

texts = text_splitter.split_text(test_all)

print(len(texts))

for text in texts:
  print(text[:10], ":", len(text))



6
제1장: 데이터 프 : 283
제2장: 울프 코퍼 : 163
제3장: 배신과 재 : 274
제4장: 울프 코퍼 : 207
제5장: 결전의 순 : 295
제7장: 새로운 시 : 191


In [14]:
!pip install faiss-cpu

In [22]:
docsearch = FAISS.from_texts(
  texts=texts, #Chunk array
  embedding=BedrockEmbeddings() #embedding
)

In [23]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm=bedrock,
  chain_type="stuff",
  retriever=docsearch.as_retriever()
)

In [24]:
print(qa_chain.run("미코의 소꿉친구 이름은?"))

 제3장에 따르면, 미코의 어릴 적 친구이자 해커 집단 일원인 료가 미코의 소꿉친구입니다. 따라서 정답은 료입니다.


In [25]:
metadatas=[
  {"source": "chapter 1"},
  {"source": "chapter 2"},
  {"source": "chapter 3"},
  {"source": "chapter 4"},
  {"source": "chapter 5~6"},
  {"source": "chapter 7"}
]


In [26]:
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores.faiss import FAISS

# Create Vector database
docsearch = FAISS.from_texts(
  texts=texts,
  embedding=BedrockEmbeddings(),
  metadatas=metadatas
)

In [28]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import Bedrock

llm_bedrock = Bedrock(
    credentials_profile_name="default", 
    model_id="anthropic.claude-v2"
)

qa_chain_source = RetrievalQAWithSourcesChain.from_chain_type(
  llm=llm_bedrock,
  chain_type="stuff",
  retriever=docsearch.as_retriever()
)

print(qa_chain_source({"question" : "미코의 친구 이름은?"}))


{'question': '미코의 친구 이름은?', 'answer': ' 문서 내용에 따르면 미코의 친구 이름은 료입니다.\n\n', 'sources': ''}


In [43]:
#SummarizeChain 
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

In [52]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import BedrockChat
from langchain.prompts import PromptTemplate

llm_bchat = BedrockChat(credentials_profile_name="default", model_id="anthropic.claude-v2")

chain_summ = load_summarize_chain(
  llm=llm_bchat,
  chain_type="refine"
)

chain_summ.run(docs)

' Here is a revised summary incorporating the new context:\n\nIn a neon-lit futuristic Tokyo, a young girl named Miko works as a courier delivering illegal data to earn money for her sick mother\'s treatment. One day she is tasked with delivering data exposing the Ulph Corporation\'s oppressive control over citizens. On her way to deliver the data to a bar called "Grandma\'s House," Miko is chased by Ulph Corporation agents who have heard rumors about the "Red Cloak" data carrier and try to intercept it from her. Miko cunningly shakes off the agents and arrives at the bar, where she meets her childhood friend Ryo who has betrayed her by joining the hackers fighting Ulph Corporation. Despite Ryo\'s anger, Miko gives him the data, deciding to join forces with Ryo to take down Ulph Corporation and free the citizens from its control. Together with the hacker group, Miko and Ryo launch a final battle against Ulph Corporation, using their hacking skills and physical abilities to penetrate th

In [41]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import BedrockChat
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs_web = loader.load()

llm = BedrockChat(credentials_profile_name="default", model_id="anthropic.claude-v2")
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

' The passage provides an overview of systems for building autonomous agents powered by large language models (LLMs). Key points:\n\n- LLMs can function as the agent\'s "brain" for planning, memory, and tool use. \n\n- Planning involves task decomposition into subgoals and self-reflection to improve. Methods like chain of thought prompt step-by-step reasoning.\n\n- Memory includes short-term (in-context learning) and long-term (vector store retrieved via algorithms like MIPS).\n\n- Tool use augments LLMs with external APIs and execution capabilities. Examples utilize search engines, calculators, etc.\n\n- Case studies demonstrate scientific discovery agents and multi-agent simulations. \n\n- Challenges include limited context, long-term planning, unreliable natural language interface.'

In [55]:
from langchain.document_loaders import PyPDFLoader


pdfread = PyPDFLoader("../data/CM5117_20220728.pdf")
docs_pdf = loader.load()

chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs_pdf)



' Here is a concise summary of the article on LLM-powered autonomous agents:\n\nThe article provides an overview of building agents with large language models (LLMs) as the core controller. Key components of such an agent system include planning mechanisms like task decomposition and self-reflection, memory systems with short-term context and long-term external stores, and tool use by integrating external APIs and models. Case studies demonstrate agents applied to scientific discovery, multi-agent simulations, and other proof-of-concept use cases. Challenges include limited context size, lack of long-term planning capabilities, unreliable natural language interfaces between the LLM and other components. Overall, the article discusses promising progress in augmenting LLMs with additional capabilities to create intelligent autonomous agents, though reliability and planning issues persist.'

In [ ]:
import pypdf as pp 

readpdf = pp.PdfReader("../data/CM5117_20220728.pdf")

print(readpdf.pages[0].extract_text())

꼼꼼하고 든든한 도로 위의 매니저
하이카자동차보험개인용자동차보험  약관2022. 07. 28 개정 CM
자동차상품파트 준법감시인 확인필 제20221957호(유효기간 2022.6.21~2023.6.20)03183  서울시 종로구 세종대로 163  현대해상빌딩
대표전화:1899-6782  | direct. hi.co.kr


In [2]:
import torch.nn as nn
import torch.optim as optim 

input_dim = 2
lr = 0.001

perceptron = optim.Perceptron(input_dim=input_dim)
bce_loss = nn.BCELoss()

optimizer = optim.Adam(perceptron.parameters(), lr=lr )


AttributeError: module 'torch.optim' has no attribute 'Perceptron'